In [4]:
from collections import defaultdict


class Graph(object):
    """ Graph data structure, undirected by default. """

    def __init__(self, connections, node_weight, directed=False):
        self._graph = defaultdict(set)
        self._directed = directed
        self.add_connections(connections)
        self.node_weight = dict()
        self.edge_price = dict()
        for idx,w in enumerate(node_weight):
            self.node_weight[idx+1] = w
    
    def increase_edge(self,edge):
        a = edge[0]
        b = edge[1]
        pa = self.node_weight[a]
        pb = self.node_weight[b]
        if pa<pb:
            inc_price = pa
        else:
            inc_price = pb
        if edge not in self.edge_price.keys():
            self.edge_price[edge] = 0
        self.edge_price[edge] += inc_price
        self.node_weight[a] -= inc_price
        self.node_weight[b] -= inc_price
        return inc_price
    
    def is_tight(self, node):
        '''
        p_sum = 0
        ad_edge = list()
        for n in g.__graph[node]:
            ad_edge.add({node,n})
        for edge in ad_edge:
            a = edge.pop()
            b = edge.pop()
            if b<a:
                tmp = b
                b = a
                a = tmp
            p_sum += price[(a,b)]
        '''
        if self.node_weight[node] == 0:
            return True
        else:
            return False

    def add_connections(self, connections):
        """ Add connections (list of tuple pairs) to graph """

        for node1, node2 in connections:
            self.add(node1, node2)

    def add(self, node1, node2):
        """ Add connection between node1 and node2 """

        self._graph[node1].add(node2)
        if not self._directed:
            self._graph[node2].add(node1)

    def remove(self, node):
        """ Remove all references to node """

        for n, cxns in self._graph.iteritems():
            try:
                cxns.remove(node)
            except KeyError:
                pass
        try:
            del self._graph[node]
        except KeyError:
            pass

    def is_connected(self, node1, node2):
        """ Is node1 directly connected to node2 """

        return node1 in self._graph and node2 in self._graph[node1]

    def find_path(self, node1, node2, path=[]):
        """ Find any path between node1 and node2 (may not be shortest) """

        path = path + [node1]
        if node1 == node2:
            return path
        if node1 not in self._graph:
            return None
        for node in self._graph[node1]:
            if node not in path:
                new_path = self.find_path(node, node2, path)
                if new_path:
                    return new_path
        return None

    def __str__(self):
        return '{}({})'.format(self.__class__.__name__, dict(self._graph))

In [5]:
def vertex_cover_approx(graph_mat, w):
    '''
    % *********************************************************************** %
    % Vertex Cover Approximation Algorithm (Pricing Method).
    % -------
    % INPUT :
    % -------
    %   graph_mat : list[tuple(2)*edge_num], Integer :: each row denotes an edge
    %       e.g., [1 2; 2 3; 3 4; 4 1; 2 4]
    %   w : list[vertex_num], Double :: node w
    %       e.g., [3 4 3 5]
    % -------
    % OUTPUT:
    % -------
    %   vertex_set_indices : matrix(1, ?), Integer :: vertex set indices generated sequentially
    %       e.g., [1 2 3]
    %   prices : matrix(1, ?), Double :: prices generated sequentially
    %       e.g., [3 1 2]
    % *********************************************************************** %
    '''
    g = Graph(list(graph_mat), w)
    uncovered = graph_mat
    price = list()
    tight_vertices = list()
    while len(uncovered)>0:
        to_covered = set()
        for edge in uncovered:
            if edge not in to_covered and (not g.is_tight(edge[0]) or  not g.is_tight(edge[1])):
                # print(edge)
                price.append(g.increase_edge(edge))
            if g.is_tight(edge[0]):
                tight_vertices.append(edge[0])
                for n2 in g._graph[edge[0]]:
                    if (n2,edge[0]) in uncovered:
                        to_covered.add((n2,edge[0]))
                    if (edge[0],n2) in uncovered:
                        to_covered.add((edge[0],n2))
            if g.is_tight(edge[1]):
                tight_vertices.append(edge[1])
                for n2 in g._graph[edge[1]]:
                    if (n2,edge[1]) in uncovered:
                        to_covered.add((n2,edge[1]))
                    if (edge[1],n2) in uncovered:
                        to_covered.add((edge[1],n2))    
        uncovered = uncovered.difference(to_covered)
    return (tight_vertices,price)

In [46]:

def vertex_cover_LP(graph_mat, w):
    '''
    % *********************************************************************** %
    % Vertex Cover Approximation Algorithm (Pricing Method).
    % -------
    % INPUT :
    % -------
    %   graph_mat : list[tuple(2)*edge_num], Integer :: each row denotes an edge
    %       e.g., [1 2; 2 3; 3 4; 4 1; 2 4]
    %   w : list[vertex_num], Double :: node w
    %       e.g., [3 4 3 5]
    % -------
    % OUTPUT:
    % -------
    %   vertex_set_indices : matrix(1, ?), Integer :: vertex set indices generated sequentially
    %       e.g., [1 2 3]
    %   prices : matrix(1, ?), Double :: prices generated sequentially
    %       e.g., [3 1 2]
    % *********************************************************************** %
    '''
    import numpy as np
    from scipy.optimize import linprog
    w = np.array(w)
    num_vertices = len(w)
    num_edges = len(graph_mat)
    A = np.zeros((num_edges,num_vertices))
    for idx, edge in enumerate(graph_mat):
        A[idx,edge[0]-1] = 1
        A[idx,edge[1]-1] = 1
    x0_bounds = np.zeros(num_vertices)
    x1_bounds = np.ones(num_vertices)
    ones = np.ones(num_edges)
    # print(len(x1_bounds), len(w))
    # print(w,-A.transpose())
    # res = linprog(w, A_ub=-A, b_ub=-ones, bounds=list(zip(x0_bounds, x1_bounds)),options={"disp": True})
    return A,w,ones,x0_bounds,x1_bounds

In [40]:
import numpy as np
def def_var(vector_len, solver, bounds):
    var = list()
    for idx,bound in zip(range(vector_len),bounds):
        var.append(solver.IntVar(bound[0],bound[1], 'x%d' % idx))
    return var

def constraint_process(A, b, solver, var):
    '''
    Ax<=b
    input: A: matrix mxn
    b: vector n
    '''
    constraint = list()
    for row,value in zip(A,b):
        print(row,value)
        constraint_tmp = solver.Constraint(-solver.infinity(), value)
        constraint.append(constraint_tmp)
        for v,var_ in zip(row,var):
            print(v,var_,type(var_))
            constraint_tmp.SetCoefficient(var_, v)
    return constraint
    # np.apply_along_axis(lambda x:print(x), axis=1, arr=A)
    
def set_obj(w,solver,var):
    '''
    minimize wTx
    '''
    objective = solver.Objective()
    for var_, weight in zip(var, w):
        print(var_,weight)
        objective.SetCoefficient(var_, weight)
    return objective

In [47]:
graph_mat = set([(1,2),(2,3),(3,4),(4,1),(2,4)])
w = [3,4,3,5]
A,w,b,x0_bounds,x1_bounds = vertex_cover_LP(graph_mat, w)

In [48]:
solver = pywraplp.Solver('SolveIntegerProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
var = def_var(len(w),solver,list(zip(x0_bounds, x1_bounds)))
constraint_process(A, b, solver, var)
set_obj(w,solver,var)

[ 1.  1.  0.  0.] 1.0
1.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>
[ 1.  0.  0.  1.] 1.0
1.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>
[ 0.  0.  1.  1.] 1.0
0.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>
[ 0.  1.  1.  0.] 1.0
0.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
1.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>


TypeError: in method 'Objective_SetCoefficient', argument 3 of type 'double'

In [35]:
constraint_process(np.zeros((4,10)), np.ones(10), solver,var)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] 1.0
0.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x4 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x5 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x6 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x7 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x8 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x9 <class 'ortools.linear_solver.pywraplp.Variable'>
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] 1.0
0.0 x0 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x1 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x2 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x3 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x4 <class 'ortools.linear_solver.pywraplp.Variable'>
0.0 x5 <class 'ortools.linear_solver.pywraplp.Variabl

[<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc65c276120> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc64c438a20> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc64c387f60> >,
 <ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7fc64c387f90> >]

In [3]:
from __future__ import print_function
from ortools.linear_solver import pywraplp



def main():
  # Instantiate a mixed-integer solver, naming it SolveIntegerProblem.
  solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  # x and y are integer non-negative variables.
  x = solver.IntVar(0.0, solver.infinity(), 'x')
  y = solver.IntVar(0.0, solver.infinity(), 'y')

  # x + 7 * y <= 17.5
  constraint1 = solver.Constraint(-solver.infinity(), 17.5)
  constraint1.SetCoefficient(x, 1)
  constraint1.SetCoefficient(y, 7)

  # x <= 3.5
  constraint2 = solver.Constraint(-solver.infinity(), 3.5)
  constraint2.SetCoefficient(x, 1)
  constraint2.SetCoefficient(y, 0)

  # Maximize x + 10 * y.
  objective = solver.Objective()
  objective.SetCoefficient(x, 1)
  objective.SetCoefficient(y, 10)
  objective.SetMaximization()

  """Solve the problem and print the solution."""
  result_status = solver.Solve()
  # The problem has an optimal solution.
  assert result_status == pywraplp.Solver.OPTIMAL

  # The solution looks legit (when using solvers other than
  # GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
  assert solver.VerifySolution(1e-7, True)

  print('Number of variables =', solver.NumVariables())
  print('Number of constraints =', solver.NumConstraints())

  # The objective value of the solution.
  print('Optimal objective value = %d' % solver.Objective().Value())
  print()
  # The value of each variable in the solution.
  variable_list = [x, y]

  for variable in variable_list:
    print('%s = %d' % (variable.name(), variable.solution_value()))

main()

Number of variables = 2
Number of constraints = 2
Optimal objective value = 23

x = 3
y = 2


In [7]:
vertex_cover_approx(graph_mat,w)

([1, 1, 3, 3, 2], [3, 3, 1])

In [8]:
graph_mat = set([(1,2),(1,3),(1,6),(2,4),(2,5),(3,4),(3,5),(4,6),(5,6)])
weights = list(range(11,0,-2))
vertex_cover_approx(graph_mat,weights)

([2, 1, 6, 6, 1, 6, 2, 4, 2, 4, 3], [9, 2, 1, 4, 1])

In [9]:
graph_mat =set()
for i in range(2,1002):
    graph_mat.add((1,i))
weights = [500,] + [1,]*1000
tight_vertices,price = vertex_cover_approx(graph_mat,weights)
print(tight_vertices,price)

[661, 344, 953, 620, 215, 816, 507, 655, 370, 518, 201, 810, 413, 64, 673, 276, 568, 227, 972, 439, 122, 859, 270, 722, 133, 998, 425, 28, 893, 288, 756, 191, 920, 579, 54, 791, 474, 750, 337, 946, 613, 40, 777, 508, 640, 331, 543, 194, 803, 406, 89, 698, 365, 561, 228, 965, 392, 115, 860, 263, 555, 158, 418, 21, 886, 313, 717, 176, 913, 580, 15, 872, 467, 743, 170, 907, 638, 33, 770, 501, 665, 332, 941, 528, 219, 804, 495, 82, 691, 358, 522, 253, 990, 385, 116, 853, 280, 556, 151, 443, 110, 847, 306, 710, 137, 605, 865, 468, 760, 163, 908, 631, 58, 795, 462, 658, 325, 934, 617, 220, 829, 480, 692, 383, 515, 246, 983, 410, 77, 686, 273, 549, 232, 969, 444, 103, 832, 267, 735, 130, 995, 598, 25, 890, 301, 753, 164, 901, 584, 51, 796, 455, 747, 350, 959, 610, 213, 822, 505, 653, 368, 516, 207, 808, 403, 70, 679, 362, 574, 225, 962, 437, 120, 857, 268, 720, 155, 996, 431, 18, 883, 294, 714, 189, 926, 577, 52, 789, 472, 748, 343, 944, 635, 46, 783, 498, 646, 329, 938, 541, 192, 801, 404, 9